THIS IS TO FIND THE FAIRNESS AND GOODNESS FOR AMAZON DATA FOR FIRST 50000 RECORDS using with the slight modication to the EDGE_DATA CODE

In [1]:
import networkx as nx
import math
import pandas as pd
from prettytable import PrettyTable
import csv

def initialize_scores(G):
    fairness = {}
    goodness = {}
    
    reviewers = set()
    products = set()
    
    for u, v, attr in G.edges(data=True):
        reviewers.add(u)
        products.add(v)
    
    for node in reviewers:
        fairness[node] = 1

    for node in products:
        try:
            goodness[node] = G.in_degree(node, weight='weight') * 1.0 / G.in_degree(node)
        except:
            goodness[node] = 0
            
    return fairness, goodness

def compute_fairness_goodness(G):
    fairness, goodness = initialize_scores(G)
    
    reviewers = set()
    products = set()
    
    for u, v, attr in G.edges(data=True):
        reviewers.add(u)
        products.add(v)
    
    iter = 0
    while iter < 100:
        df = 0
        dg = 0

        print('-----------------')
        print("Iteration number", iter)

        print('Updating goodness')
        for node in products:
            inedges = G.in_edges(node, data='weight')
            g = 0
            for edge in inedges:
                if isinstance(edge[2], float):
                    g += fairness[edge[0]] * edge[2]
                else:
                    g += fairness[edge[0]] * edge[2]["weight"]

            try:
                dg += abs(g / len(inedges) - goodness[node])
                goodness[node] = g / len(inedges)
            except:
                pass

        print('Updating fairness')
        for node in reviewers:
            outedges = G.out_edges(node, data='weight')
            f = 0
            for edge in outedges:
                if isinstance(edge[2], float):
                    f += 1.0 - abs(edge[2] - goodness[edge[1]]) / 2.0
                else:
                    f += 1.0 - abs(edge[2]["weight"] - goodness[edge[1]]) / 2.0
            try:
                df += abs(f / len(outedges) - fairness[node])
                fairness[node] = f / len(outedges)
            except:
                pass

        print('Differences in fairness score and goodness score = %.2f, %.2f' % (df, dg))
        if df < math.pow(10, -6) and dg < math.pow(10, -6):
            break
        iter += 1

    return fairness, goodness

G = nx.DiGraph()

"---------------------------------------------------------------------------------------------------------------------------------"

with open("amazon.csv", "r") as f:
    csvreader = csv.reader(f)
    for row in csvreader:
        G.add_edge(row[0], row[1], weight=float(row[2]))

fairness, goodness = compute_fairness_goodness(G)

reviewers = set()
products = set()

for u, v, attr in G.edges(data=True):
    reviewers.add(u)
    products.add(v)

result_table = PrettyTable()
result_table.field_names = ["Reviewer", "Fairness", "Product", "Goodness"]

for reviewer in reviewers:
    result_table.add_row([reviewer, fairness[reviewer], "", ""])

for product in products:
    result_table.add_row(["", "", product, goodness[product]])

print(result_table)

-----------------
Iteration number 0
Updating goodness
Updating fairness
Differences in fairness score and goodness score = 8883.43, 0.00
-----------------
Iteration number 1
Updating goodness
Updating fairness
Differences in fairness score and goodness score = 12668.53, 5182.67
-----------------
Iteration number 2
Updating goodness
Updating fairness
Differences in fairness score and goodness score = 21095.92, 7030.87
-----------------
Iteration number 3
Updating goodness
Updating fairness
Differences in fairness score and goodness score = 44285.59, 14132.02
-----------------
Iteration number 4
Updating goodness
Updating fairness
Differences in fairness score and goodness score = 91564.23, 29418.27
-----------------
Iteration number 5
Updating goodness
Updating fairness
Differences in fairness score and goodness score = 189752.45, 61256.54
-----------------
Iteration number 6
Updating goodness
Updating fairness
Differences in fairness score and goodness score = 393964.16, 127643.12
---

In [2]:
highest_fairness_value = max(fairness.values())
lowest_fairness_value = min(fairness.values())

highest_fairness_nodes = [node for node, value in fairness.items() if value == highest_fairness_value]
lowest_fairness_nodes = [node for node, value in fairness.items() if value == lowest_fairness_value]

print("Nodes with highest fairness:", highest_fairness_nodes)
print("Fairness value:", highest_fairness_value)
print("Nodes with lowest fairness:", lowest_fairness_nodes)
print("Fairness value:", lowest_fairness_value)

print("\n")

highest_goodness_value = max(goodness.values())
lowest_goodness_value = min(goodness.values())

highest_goodness_nodes = [node for node, value in goodness.items() if value == highest_goodness_value]
lowest_goodness_nodes = [node for node, value in goodness.items() if value == lowest_goodness_value]

print("Nodes with highest goodness:", highest_goodness_nodes)
print("Goodness value:", highest_goodness_value)
print("Nodes with lowest goodness:", lowest_goodness_nodes)
print("Goodness value:", lowest_goodness_value)

Nodes with highest fairness: ['A1LXH8HJ37UUDJ', 'A16ME5K5S0WJE7', 'ANBGGR4671OM7', 'A1VLW4UBDI185B', 'ACO1YFQC66OZE', 'A2A5F4SSO4DVVA', 'A2D705WKMKTU9I', 'A21AUK3OY0RSYX', 'A1NQJDDJUFEB2A', 'A1EHOSKJ00XN5A', 'A2BFZUWQB64ISB', 'AAVZOTZWVQYRV', 'A1HUO1KUCRLWGC', 'A12TVQ5C78477Q', 'A1MRFHG069EZPU', 'A2SVNXI763GZQF', 'A397FW6DFA6LV2', 'A1OK0GOWE9II6R', 'A1TNRTUZI9Z55Z', 'A2WSUIDV7G55KT', 'A1Z99QU9GCTRGS', 'A1DKEAJ7NAD6D1', 'A2CE0RG8F5CJB', 'A2QS7M3MH1D206', 'A2BFJ452QA0AX9', 'A16VDEDYH39QSB']
Fairness value: 1.0
Nodes with lowest fairness: ['A2ISXZMZK2X643', 'A3CFACM6X74OOJ', 'A25L6X0XT9IXEP', 'A33AG35TJLBVVZ', 'A2OPZ4IUPQ8933', 'A1KZVXCQXJ51D8']
Fairness value: -1.3134713281670688e+37


Nodes with highest goodness: ['B004NIFW4O', 'B004M1913U', 'B00480OPAY', 'B004M5HKY8', 'B003Z0CZKU', 'B004BDP5WA', 'B004OR1BYY']
Goodness value: 5.0
Nodes with lowest goodness: ['B0023RTBB2']
Goodness value: -2.6269426563341376e+37
